In [12]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

def scrap(año, mes):
    url  = 'https://www.cotizacion-dolar.com.ar/dolar-blue-historico-'+str(año)+'.php'
    for i in range(1,7):
        try:        
            fecha = datetime.datetime(año,mes,i)
            data = {'fecha': fecha.strftime('%d-%m-%y')}
            resp = requests.post(url, data=data)
            soup = BeautifulSoup(resp.text, "html.parser")
            break
        except:
            print('Falló en ',i)    
    filas = soup.find_all('td', {'style' : 'padding: 1%'})
    return filas

def parsear(filas):
    mensual = pd.DataFrame() 
    for i in range(1, int(len(list(filas))/3)):
        dic = {}
        dic['fecha'] = filas[3*i].text
        dic['bid'] = filas[3*i+1].text
        dic['ask'] = filas[3*i+2].text
        rueda = pd.DataFrame.from_dict(dic, orient='index').transpose().set_index('fecha')
        rueda.index = pd.to_datetime(rueda.index, format='%d-%m-%y ')
        mensual = pd.concat([mensual,rueda], axis=0)
    return mensual

def downloadAño(año):
    tablaAnual = pd.DataFrame()
    for i in range(1,13):
        filas = scrap(año=año, mes=i)
        tabla = parsear(filas)
        tablaAnual = pd.concat([tablaAnual,tabla],axis=0)
        print('mes',i,'listo')
        
    tablaAnual.to_csv('blue_'+str(año)+'.csv')  # Cambio de .to_excel a .to_csv
    print(tablaAnual)

downloadAño(2023)


mes 1 listo
mes 2 listo
mes 3 listo
Falló en  1
mes 4 listo
mes 5 listo
mes 6 listo
mes 7 listo
mes 8 listo
mes 9 listo
mes 10 listo
mes 11 listo
mes 12 listo
                   bid        ask
fecha                            
2023-01-02   $ 342.00    $ 346.00
2023-01-03   $ 350.00    $ 354.00
2023-01-04   $ 350.00    $ 354.00
2023-01-05   $ 348.00    $ 353.00
2023-01-06   $ 350.00    $ 354.00
...                ...        ...
2023-08-14   $ 665.00    $ 685.00
2023-08-15   $ 720.00    $ 730.00
2023-08-16   $ 770.00    $ 780.00
2023-08-17   $ 750.00    $ 760.00
2023-08-18   $ 710.00    $ 720.00

[163 rows x 2 columns]
